In [52]:
import torch
import importlib
import doctest

In [53]:
from networks.gradops import gradops_2
importlib.reload(gradops_2)
gradops_2_doctest_result = doctest.testmod(gradops_2)
print(gradops_2_doctest_result)

u.shape: torch.Size([4, 4])
**********************************************************************
File "/mnt/h/GIT/DISSERTATION/LearningRegularizationParameterMaps/networks/gradops/gradops_2.py", line 74, in networks.gradops.gradops_2.dx_backward
Failed example:
    dx_backward(u)
Expected:
    tensor([[ 1.,  2.,  3.],
            [ 3.,  3.,  3.],
            [-4., -5., -6.]])
Got:
    u.shape: torch.Size([3, 3])
    tensor([[ 1.,  2.,  3.],
            [ 3.,  3.,  3.],
            [-4., -5., -6.]])
**********************************************************************
File "/mnt/h/GIT/DISSERTATION/LearningRegularizationParameterMaps/networks/gradops/gradops_2.py", line 82, in networks.gradops.gradops_2.dx_backward
Failed example:
    dx_backward(u)
Expected:
    tensor([[[ 1.,  2.,  3.],
             [ 3.,  3.,  3.],
             [-4., -5., -6.]]])
Got:
    u.shape: torch.Size([1, 3, 3])
    tensor([[[ 1.,  2.,  3.],
             [ 3.,  3.,  3.],
             [-4., -5., -6.]]])
*****

In [54]:
from networks import tgv_pdhg_net
importlib.reload(tgv_pdhg_net)
from networks.tgv_pdhg_net import TgvPdhgNet

In [55]:
net = TgvPdhgNet(
    cnn=None,
    device="cuda" if torch.cuda.is_available() else "cpu",
    constraint_activation="softplus",
    upper_bound=1.0,
)

convergence_limit: 0.08793310821056366


In [56]:
def test_scalar():
    alpha0 = 0.1
    alpha1 = 0.1
    params = (alpha0, alpha1)
    u = torch.randn(1, 1, 512, 512)
    T = 128
    u_T = net(u=u, regularisation_params=params, T=T)
    print(f"u_T.shape = {u_T.shape}")

In [57]:
test_scalar()

u_T.shape = torch.Size([1, 1, 512, 512])
